Getting the corect description, main actors and more like this section which are the related films
(i stopped it at the beggining just to check if the first films info was correct)

In [53]:
import time
import pandas as pd
from selenium import webdriver
from bs4 import BeautifulSoup
import re

# Load the list of movies from the CSV file
file_path = 'C:/Users/Alba/Desktop/MASTER/4_SEMESTER/Social Graphs/FINAL/movies_info.csv'
df = pd.read_csv(file_path)

# List to store the scraped data
movie_data = []

# Set up Chrome options to force English language
options = webdriver.ChromeOptions()
options.add_argument("--lang=en")  # Set the browser language to English

# Initialize the WebDriver with the options
driver = webdriver.Chrome(options=options)  # Make sure ChromeDriver is in your PATH

# Function to scrape details from each movie page
def scrape_movie_details(url):
    driver.get(url)
    time.sleep(3)  # Allow time for the page to load
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    
    # Extract details using the same method for JSON-LD data (Director extraction)
    try:
        # Extract the JSON-LD script
        json_ld = driver.execute_script('return document.querySelector("script[type=\'application/ld+json\']").innerText')
        
        # Extract the director from the JSON-LD data
        director = json_ld.split('"director":')[1].split('}')[0].split('"name":"')[1].split('"')[0].strip()
    except:
        director = None
    
    # Extract other details as needed
    try:
        title = soup.find('h1').text.strip()
    except:
        title = None

    # Extract 'More Like This' movies (Related films)
    try:
        more_like_this = [a.text.strip() for a in soup.select('.ipc-poster-card__title')]
    except:
        more_like_this = None
    
    try:
        # Extract full description including director and actors
        description = soup.find('meta', {'name': 'description'})['content']
    except:
        description = None

    # If we have a description, process it
    if description:
        # Split description into sentences
        description_parts = description.split('.')
        
        # Extract the director from the first sentence
        director_info = description_parts[0] if len(description_parts) > 0 else ""
        director_match = re.search(r'Directed by\s*([^\.]+)', director_info)
        if director_match:
            director = director_match.group(1).strip()

        # Extract actors from the second sentence
        actors = []
        if len(description_parts) > 1:
            second_sentence = description_parts[1].strip()  # The second sentence with actors

            # Extract actors after "With" or "Starring"
            actor_match = re.search(r'(With|Starring)\s*([^\.]+)', second_sentence)
            if actor_match:
                actors = [actor.strip() for actor in actor_match.group(2).split(',')]
        
        # Clean the description by removing the director and actors parts
        clean_description = '.'.join(description_parts[2:]).strip() if len(description_parts) > 2 else ""

    # Append data to the movie_data list with try-except to avoid errors disrupting the loop
    try:
        movie_data.append({
            'Title': title,
            'URL': url,
            'Description': clean_description,
            'Actors': actors,
            'More Like This': more_like_this,  
        })
    except Exception as e:
        print(f"Error appending data for URL: {url} - {e}")

# Iterate through each movie URL in the dataset and scrape details
for url in df['URL']:
    scrape_movie_details(url)

# Convert the list of dictionaries to a DataFrame and save to CSV
df_movies = pd.DataFrame(movie_data)
df_movies.to_csv('scraped_movie_data.csv', index=False)

# Close the Selenium browser
driver.quit()

print("Data scraping completed and saved to 'scraped_movie_data.csv'.")


KeyboardInterrupt: 

In [54]:
# Convert the list of dictionaries to a DataFrame and save to CSV
df_movies = pd.DataFrame(movie_data)
df_movies.to_csv('scraped_movie_data.csv', index=False)

# Close the Selenium browser
driver.quit()

print("Data scraping completed and saved to 'scraped_movie_data.csv'.")


Data scraping completed and saved to 'scraped_movie_data.csv'.


Getting the description actors and more like this and adding it to the actual csv to have all the info together for the first 10 films just to check!!!!

In [55]:
import time
import pandas as pd
from selenium import webdriver
from bs4 import BeautifulSoup
import re
import urllib.parse

# Load the existing CSV file
file_path = 'C:/Users/Alba/Desktop/MASTER/4_SEMESTER/Social Graphs/FINAL/movies_info_copy.csv'
df_existing = pd.read_csv(file_path)

# List to store the scraped data (will be appended to the existing DataFrame)
new_data = []

# Set up Chrome options to force English language
options = webdriver.ChromeOptions()
options.add_argument("--lang=en")  # Set the browser language to English

# Initialize the WebDriver with the options
driver = webdriver.Chrome(options=options)  # Make sure ChromeDriver is in your PATH

# Function to scrape details from each movie page
def scrape_movie_details(url):
    # Ensure the URL is well-formed
    if not url or not isinstance(url, str):
        print(f"Skipping invalid URL: {url}")
        return
    
    # Validate if the URL is a proper URL format (check if it starts with http/https)
    if not urllib.parse.urlparse(url).scheme:
        print(f"Skipping malformed URL: {url}")
        return
    
    try:
        driver.get(url)
        time.sleep(3)  # Allow time for the page to load
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        
        # Extract 'More Like This' movies
        try:
            more_like_this = [a.text.strip() for a in soup.select('.ipc-poster-card__title')]
        except:
            more_like_this = None
        
        try:
            # Extract full description including director and actors
            description = soup.find('meta', {'name': 'description'})['content']
        except:
            description = None

        # If we have a description, process it
        if description:
            # Split description into sentences
            description_parts = description.split('.')
            
            # Extract the director from the first sentence
            director_info = description_parts[0] if len(description_parts) > 0 else ""
            director_match = re.search(r'Directed by\s*([^\.]+)', director_info)
            if director_match:
                director = director_match.group(1).strip()

            # Extract actors from the second sentence
            actors = []
            if len(description_parts) > 1:
                second_sentence = description_parts[1].strip()  # The second sentence with actors

                # Extract actors after "With" or "Starring"
                actor_match = re.search(r'(With|Starring)\s*([^\.]+)', second_sentence)
                if actor_match:
                    actors = [actor.strip() for actor in actor_match.group(2).split(',')]
            
            # Clean the description by removing the director and actors parts
            clean_description = '.'.join(description_parts[2:]).strip() if len(description_parts) > 2 else ""


        # Append data to the new_data list (in the format to match the existing CSV)
        try:
            new_data.append({
                'URL': url,
                'Description_good': clean_description,
                'Actors': actors,
                'More Like This': more_like_this,  
            })
        except Exception as e:
            print(f"Error appending data for URL: {url} - {e}")

    except Exception as e:
        print(f"Error processing URL {url}: {e}")

# Limit the iteration to the first 10 films
urls_to_scrape = df_existing['URL'][:10]

# Iterate through each movie URL in the first 10 rows and scrape details
for url in urls_to_scrape:
    scrape_movie_details(url)

# Convert the new data list to a DataFrame
df_new_data = pd.DataFrame(new_data)

# Merge the new data with the existing data using the 'URL' column as the key
df_updated = pd.merge(df_existing, df_new_data, on='URL', how='left')

# Save the updated DataFrame back to the same CSV file (you can change the filename if needed)
df_updated.to_csv(file_path, index=False)

# Close the Selenium browser
driver.quit()

print("Data scraping completed and added to the existing CSV for the first 10 films.")


Data scraping completed and added to the existing CSV for the first 10 films.


Getting the description actors and more like this and adding it to the actual csv to have all the info together for all the 787 films!

In [ ]:
import time
import pandas as pd
from selenium import webdriver
from bs4 import BeautifulSoup
import re
import urllib.parse

# Load the existing CSV file
file_path = 'C:/Users/Alba/Desktop/MASTER/4_SEMESTER/Social Graphs/FINAL/movies_info_copy.csv'
df_existing = pd.read_csv(file_path)

# List to store the scraped data (will be appended to the existing DataFrame)
new_data = []

# Set up Chrome options to force English language
options = webdriver.ChromeOptions()
options.add_argument("--lang=en")  # Set the browser language to English

# Initialize the WebDriver with the options
driver = webdriver.Chrome(options=options)  # Make sure ChromeDriver is in your PATH

# Function to scrape details from each movie page
def scrape_movie_details(url):
    # Ensure the URL is well-formed
    if not url or not isinstance(url, str):
        print(f"Skipping invalid URL: {url}")
        return
    
    # Validate if the URL is a proper URL format (check if it starts with http/https)
    if not urllib.parse.urlparse(url).scheme:
        print(f"Skipping malformed URL: {url}")
        return
    
    try:
        driver.get(url)
        time.sleep(3)  # Allow time for the page to load
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        
        # Extract details using the same method for JSON-LD data (Director extraction)
        try:
            # Extract the JSON-LD script
            json_ld = driver.execute_script('return document.querySelector("script[type=\'application/ld+json\']").innerText')
            
            # Extract the director from the JSON-LD data
            director = json_ld.split('"director":')[1].split('}')[0].split('"name":"')[1].split('"')[0].strip()
        except:
            director = None
        
        # Extract other details as needed
        try:
            title = soup.find('h1').text.strip()
        except:
            title = None

        # Extract 'More Like This' movies (Related films)
        try:
            more_like_this = [a.text.strip() for a in soup.select('.ipc-poster-card__title')]
        except:
            more_like_this = None
        
        try:
            # Extract full description including director and actors
            description = soup.find('meta', {'name': 'description'})['content']
        except:
            description = None

        # If we have a description, process it
        if description:
            # Split description into sentences
            description_parts = description.split('.')
            
            if len(description_parts) > 1:
                second_sentence = description_parts[1].strip()  # Get the second sentence (actors info)

                # Extract actors from the second sentence (assuming the format includes 'Starring' or 'Starring: ' before actors' names)
                actors = re.findall(r'(?:Starring|Con):?\s*([^\.]+)', second_sentence)

                # Clean up the list of actors (if needed)
                actors = [actor.strip() for actor in actors[0].split(',')] if actors else []

                # Remove the second sentence (actors' names) from the description
                clean_description = '.'.join(description_parts[2:]).strip() if len(description_parts) > 2 else ""
            
            else:
                clean_description = description.strip()
                actors = []

        # Append data to the new_data list (in the format to match the existing CSV)
        try:
            new_data.append({
                'URL': url,
                'Description_good': clean_description,
                'Actors': actors,
                'More Like This': more_like_this,  
            })
        except Exception as e:
            print(f"Error appending data for URL: {url} - {e}")

    except Exception as e:
        print(f"Error processing URL {url}: {e}")

# Iterate through each movie URL in the existing dataset and scrape details
for url in df_existing['URL']:
    scrape_movie_details(url)

# Convert the new data list to a DataFrame
df_new_data = pd.DataFrame(new_data)

# Merge the new data with the existing data using the 'URL' column as the key
df_updated = pd.merge(df_existing, df_new_data, on='URL', how='left')

# Save the updated DataFrame back to the same CSV file (you can change the filename if needed)
df_updated.to_csv(file_path, index=False)

# Close the Selenium browser
driver.quit()

print("Data scraping completed and added to the existing CSV.")
